In [10]:
import os
import sys
import time
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

os.environ['CUDA_VISIBLE_DEVICES'] = '3'

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow.keras.backend as K
from tensorboard.plugins.hparams import api as hp
from tensorflow.keras.optimizers import RMSprop, Adam, SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.layers import Dense, Dropout, InputLayer
from tensorflow.keras.regularizers import l1, l2
from sklearn.metrics import confusion_matrix

In [27]:
def gridSearchOptimize(mdlname, verbose=True):
    if not os.path.exists('../models/{}'.format(mdlname)):
        os.mkdir('../models/{}'.format(mdlname))
        
    K.clear_session() # Clear any previous session!
    iteration = 1
    res = []

    # Hyperparams
    batch_sizes = [8, 16]
    optimizers = [RMSprop, Adam, SGD]
    learn_rates = [0.001, 0.01, 0.1]    
    dropouts = [0, 0.1, 0.2]
    label_smoothening = [0, 0.1, 0.2]
    regularizers = [l1, l2]
    regularizer_values = [0.0001, 0.001]
    allow_training = [False, True]
    start_time = time.time()
    iteration_time = time.time()

    print("Started hyperparameter tuning...")
    for batchSize in batch_sizes:
        for optimizer in optimizers:
            for learnRate in learn_rates:
                for dropOutRate in dropouts:
                    for lblSmooth in label_smoothening:
                        for reg in regularizers:
                            for regVal in regularizer_values:
                                for allowTuning in allow_training:
                                    loop_time = time.time()
                                    loss, acc = np.random.rand(), np.random.rand()
                                    res.append([
                                        loss, acc, batchSize, optimizer, learnRate, dropOutRate, lblSmooth, reg, regVal, allowTuning
                                    ])
                                    if verbose and ((iteration % 8) == 0):
                                        print(iteration, "Processed ---> {:.3f} LOSS, {:.3f} ACCU (this {:.4f} sec, overall {:.4f} sec)".format(loss, acc, (time.time() - loop_time), (time.time() - iteration_time)))
                                        iteration_time = time.time()
                                    iteration += 1
                                    K.clear_session() # Clear memory used. Proceed to next!

    end_time = time.time()
    print("Finished hyperparameter tuning, time elapsed: {:.4f} sec.".format((end_time - start_time)))

    if len(res) == 0:
        print("No results were recorded?!")
        return

    res.sort(key = lambda x: x[0]) # Sort on LOSS, low->high
    print("Writing results to ../models/{}/stat.csv".format(mdlname))
    with open('../models/{}/stat.csv'.format(mdlname), 'w') as f:
        for obj in res:
            s = ','.join([str(v) for v in obj])
            f.write('{}\n'.format(s))

    print("Finished hyperparameter tuning!")

In [29]:
gridSearchOptimize('test')

Started hyperparameter tuning...
8 Processed ---> 0.020 LOSS, 0.496 ACCU (this 0.0000 sec, overall 0.0120 sec)
16 Processed ---> 0.032 LOSS, 0.344 ACCU (this 0.0000 sec, overall 0.0140 sec)
24 Processed ---> 0.381 LOSS, 0.770 ACCU (this 0.0000 sec, overall 0.0130 sec)
32 Processed ---> 0.957 LOSS, 0.889 ACCU (this 0.0000 sec, overall 0.0140 sec)
40 Processed ---> 0.879 LOSS, 0.051 ACCU (this 0.0000 sec, overall 0.0130 sec)
48 Processed ---> 0.021 LOSS, 0.715 ACCU (this 0.0000 sec, overall 0.0120 sec)
56 Processed ---> 0.510 LOSS, 0.260 ACCU (this 0.0000 sec, overall 0.0130 sec)
64 Processed ---> 0.577 LOSS, 0.574 ACCU (this 0.0000 sec, overall 0.0130 sec)
72 Processed ---> 0.230 LOSS, 0.644 ACCU (this 0.0000 sec, overall 0.0120 sec)
80 Processed ---> 0.987 LOSS, 0.771 ACCU (this 0.0000 sec, overall 0.0130 sec)
88 Processed ---> 0.205 LOSS, 0.511 ACCU (this 0.0000 sec, overall 0.0120 sec)
96 Processed ---> 0.852 LOSS, 0.646 ACCU (this 0.0000 sec, overall 0.0130 sec)
104 Processed ---> 0

848 Processed ---> 0.558 LOSS, 0.723 ACCU (this 0.0000 sec, overall 0.0130 sec)
856 Processed ---> 0.319 LOSS, 0.356 ACCU (this 0.0000 sec, overall 0.0120 sec)
864 Processed ---> 0.733 LOSS, 0.453 ACCU (this 0.0000 sec, overall 0.0130 sec)
872 Processed ---> 0.265 LOSS, 0.235 ACCU (this 0.0000 sec, overall 0.0130 sec)
880 Processed ---> 0.655 LOSS, 0.232 ACCU (this 0.0000 sec, overall 0.0130 sec)
888 Processed ---> 0.610 LOSS, 0.084 ACCU (this 0.0000 sec, overall 0.0120 sec)
896 Processed ---> 0.220 LOSS, 0.597 ACCU (this 0.0000 sec, overall 0.0120 sec)
904 Processed ---> 0.422 LOSS, 0.659 ACCU (this 0.0000 sec, overall 0.0130 sec)
912 Processed ---> 0.829 LOSS, 0.650 ACCU (this 0.0000 sec, overall 0.0130 sec)
920 Processed ---> 0.953 LOSS, 0.020 ACCU (this 0.0000 sec, overall 0.0120 sec)
928 Processed ---> 0.799 LOSS, 0.487 ACCU (this 0.0000 sec, overall 0.0130 sec)
936 Processed ---> 0.341 LOSS, 0.607 ACCU (this 0.0000 sec, overall 0.0130 sec)
944 Processed ---> 0.334 LOSS, 0.705 ACC

In [40]:
def createDF(data):
    data = np.array(data)    
    df = pd.DataFrame()
    cols = ['Loss', 'Acc', 'BatchSize', 'Optimizer', 'LearnRate', 'DropOut', 'LabelSmooth', 'Regularizer', 'RegValue', 'AllowTune']
    for i,v in enumerate(cols):
        df[v] = data[:,i]
    return df

In [41]:
content = []
with open('../models/mobilenet_fast/stat.csv', 'r') as f:
    for line in f:
        line = line.strip().lower()
        line = line.replace('tensorflow.python.keras.optimizer_v2.gradient_descent.', '').replace('tensorflow.python.keras.regularizers.', '')
        line = line.split(',')
        line[0], line[1] = float('{:.5f}'.format(float(line[0]))), float('{:.5f}'.format(float(line[1])))
        content.append(line)

In [44]:
content.sort(key=lambda x: x[1], reverse=True)
df1 = createDF(content[:10])
df1

,Loss,Acc,BatchSize,Optimizer,LearnRate,DropOut,LabelSmooth,Regularizer,RegValue,AllowTune
0,1.03656,0.68918,16,<class 'sgd'>,0.001,0,0,<class 'l1'>,0.001,false
1,1.09448,0.66683,16,<class 'sgd'>,0.001,0.2,0,<class 'l1'>,0.001,false
2,0.86586,0.65392,16,<class 'sgd'>,0.001,0.2,0,<class 'l2'>,0.0001,false
3,1.06023,0.64945,16,<class 'sgd'>,0.001,0.1,0,<class 'l1'>,0.001,false
4,0.86956,0.64499,16,<class 'sgd'>,0.001,0,0,<class 'l2'>,0.001,false
5,0.88838,0.643,16,<class 'sgd'>,0.001,0,0,<class 'l1'>,0.0001,false
6,0.88584,0.63505,16,<class 'sgd'>,0.001,0,0,<class 'l2'>,0.0001,false
7,0.90844,0.62959,16,<class 'sgd'>,0.001,0.1,0,<class 'l2'>,0.001,false
8,0.92296,0.61718,16,<class 'sgd'>,0.001,0.2,0,<class 'l1'>,0.0001,false
9,0.97155,0.61668,16,<class 'sgd'>,0.001,0.2,0,<class 'l2'>,0.001,false


In [45]:
content.sort(key=lambda x: x[0], reverse=False)
df2 = createDF(content[:10])
df2

,Loss,Acc,BatchSize,Optimizer,LearnRate,DropOut,LabelSmooth,Regularizer,RegValue,AllowTune
0,0.86586,0.65392,16,<class 'sgd'>,0.001,0.2,0,<class 'l2'>,0.0001,false
1,0.86956,0.64499,16,<class 'sgd'>,0.001,0,0,<class 'l2'>,0.001,false
2,0.88584,0.63505,16,<class 'sgd'>,0.001,0,0,<class 'l2'>,0.0001,false
3,0.88838,0.643,16,<class 'sgd'>,0.001,0,0,<class 'l1'>,0.0001,false
4,0.90844,0.62959,16,<class 'sgd'>,0.001,0.1,0,<class 'l2'>,0.001,false
5,0.92296,0.61718,16,<class 'sgd'>,0.001,0.2,0,<class 'l1'>,0.0001,false
6,0.92391,0.6137,16,<class 'sgd'>,0.001,0.1,0,<class 'l2'>,0.0001,false
7,0.95092,0.60427,16,<class 'sgd'>,0.001,0.1,0,<class 'l1'>,0.0001,false
8,0.97155,0.61668,16,<class 'sgd'>,0.001,0.2,0,<class 'l2'>,0.001,false
9,1.03656,0.68918,16,<class 'sgd'>,0.001,0,0,<class 'l1'>,0.001,false
